In [16]:
import pandas as pd
import numpy as np

# Data preparation

In [115]:
df1 = pd.read_csv('/content/drive/MyDrive/fsa_hack/data1.csv', sep="\t") 
df2 = pd.read_csv('/content/drive/MyDrive/fsa_hack/data2.csv', sep="\t") 
df1e = pd.read_csv('/content/drive/MyDrive/fsa_hack/data1_ep.csv', sep="\t") 
df2e = pd.read_csv('/content/drive/MyDrive/fsa_hack/data2_ep.csv', sep="\t") 
dfc = df1e = pd.read_csv('/content/drive/MyDrive/fsa_hack/categories.csv', sep="\t")

In [116]:
df = pd.DataFrame()

df = df2

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267684 entries, 0 to 267683
Data columns (total 4 columns):
 #   Column                                                     Non-Null Count   Dtype 
---  ------                                                     --------------   ----- 
 0   id                                                         267684 non-null  int64 
 1   Общее наименование продукции                               267684 non-null  object
 2   Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)  267684 non-null  object
 3   Подкатегория продукции                                     267684 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.2+ MB


,id,Общее наименование продукции,Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции),Подкатегория продукции
0,1,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у..."
1,2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
2,3,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...
3,4,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...
4,5,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...


In [117]:
del df2 
df = df.drop(columns = 'id')
df['category'] = df['Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)'].apply(lambda x: x.split('.')[0])
df['category'] = df['category'].apply(lambda x: int(x.split(';')[0]))
df.head()

,Общее наименование продукции,Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции),Подкатегория продукции,category
0,"Парацетамол таблетки 500 мг 10 шт., упаковки я...",9300.1,"Лекарственные средства, зарегистрированные в у...",9300
1,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,3482
2,Перезаряжаемая литий-ионная батарея торговой м...,3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,3482
3,Аппарат вакуумно-лазерной терапии стоматологич...,9444.4,Приборы и аппараты для электролечения высокоча...,9444
4,Блоки оконные и балконные дверные из алюминиев...,5270.1,Блоки оконные и балконные дверные из алюминиев...,5270


In [118]:
del df1, df1e, df2e

In [157]:
df.rename(columns={"Общее наименование продукции": "text", 
                   "Раздел ЕП РФ (Код из ФГИС ФСА для подкатегории продукции)": "subcategory",
                   "Подкатегория продукции": "subcategory_name"},
          inplace=True)

In [120]:
import re
df["text"] = df['text'].apply(lambda x: re.findall(r'[а-я]+', x.lower()))
df.head()

,text,subcategory,subcategory_name,category
0,"[парацетамол, таблетки, мг, шт, упаковки, ячей...",9300.1,"Лекарственные средства, зарегистрированные в у...",9300
1,"[перезаряжаемая, литий, ионная, батарея, торго...",3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,3482
2,"[перезаряжаемая, литий, ионная, батарея, торго...",3482.2,Аккумуляторы и аккумуляторные батареи никель-м...,3482
3,"[аппарат, вакуумно, лазерной, терапии, стомато...",9444.4,Приборы и аппараты для электролечения высокоча...,9444
4,"[блоки, оконные, и, балконные, дверные, из, ал...",5270.1,Блоки оконные и балконные дверные из алюминиев...,5270


In [ ]:
!pip install pymorphy2

In [121]:
import pymorphy2

def pos(text, morph=pymorphy2.MorphAnalyzer()):
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}
    r = []
    for w in text:
        if w not in functors_pos:
            r.append(w)
    return ' '.join(r)
df['text'] =df['text'].apply(pos)

In [122]:
df = df.astype({'subcategory': 'str',
           'subcategory_name': 'str',
           'category': 'str'}
          )

In [124]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

In [125]:
data = df

# Обучение модели
source: https://www.kaggle.com/constantinwerner/three-berts-to-solve-intent-classification

In [126]:
NUM_LABELS=len(set(data['category']))

In [127]:
labels=set(data['category'])

In [128]:
id2label = {i:label for i,label in enumerate(labels) if label != None}
label2id = {label:i for i,label in enumerate(labels) if label != None}

In [129]:
print(data['category'].value_counts())
print('examples:\n',data['text'].iloc[2]+'\n'+data['text'].iloc[929]+'\n'+data['text'].iloc[24]+'\n'+data['text'].iloc[1224])

9300    160098
2293     12508
2364     12290
3482      5014
9392      4396
         ...  
5341         1
964          1
3416         1
2322         1
9381         1
Name: category, Length: 129, dtype: int64
examples:
 перезаряжаемая литий ионная батарея торговой марки модель
ацетилсалициловая кислота таблетки мг шт упаковки ячейковые контурные рег р от дата переоформления серия партия упаковок годен до производства оао фармстандарт лексредства инн курская область курск ул я агрегатная а россия код окпд
лекарственный препарат натрия хлорид буфус растворитель для приготовления лекарственных форм для инъекций мл ампулы полимерные пачки картонные серия партия упаковок
мелаксен таблетки покрытые оболочкой мг шт упаковки ячейковые контурные пачки картонные годен до код тн вэд контракт р от инвойс от


In [33]:
!pip install datasets
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 4.9.0 requires huggingface-hub==0.0.12, but you'll have huggingface-hub 0.1.2 which is incompatible.
You should consider upgrading via the 'f:\python3.9\python.exe -m pip install --upgrade pip' command.
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.60.0
    Uninstalling tqdm-4.60.0:
      Successfully uninstalled tqdm-4.60.0
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.0.12
    Uninstalling huggingface-hub-0.0.12:
      Successfully uninstalled huggingface-hub-0.0.12
Defaulting to user installation because normal site-packages is not write

In [34]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
#MODEL_NAME = 'DeepPavlov/rubert-base-cased-sentence'
MODEL_NAME = 'cointegrated/rubert-tiny'
#MODEL_NAME = '/content/drive/MyDrive/fsa_hack/model'
N_MODELS = 1
tokenizers = []
models = []
for i in range(N_MODELS):
    tokenizers.append(AutoTokenizer.from_pretrained(MODEL_NAME))
    models.append(AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,label2id=label2id,id2label=id2label,num_labels =NUM_LABELS))

#####
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading: 100%|██████████| 632/632 [00:00<00:00, 158kB/s]
Downloading: 100%|██████████| 241k/241k [00:00<00:00, 286kB/s]
Downloading: 100%|██████████| 468k/468k [00:01<00:00, 322kB/s]
Downloading: 100%|██████████| 112/112 [00:00<00:00, 37.5kB/s]


NameError: name 'label2id' is not defined

In [142]:
import datasets as d 
from datasets import load_dataset

train_dataset = d.Dataset.from_pandas(train[['text', 'category']])
train_dataset = train_dataset.remove_columns("__index_level_0__")
test_dataset = d.Dataset.from_pandas(test[['text', 'category']])
test_dataset = test_dataset.remove_columns("__index_level_0__")

In [143]:
dataset = d.DatasetDict({"train":train_dataset,"test":test_dataset})

In [144]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize(batch):
    tokenized_batch = tokenizer(batch['text'], padding=True, truncation=True)
    tokenized_batch["label"] = [label2id[label] for label in batch["category"]]
    return tokenized_batch

loading configuration file https://huggingface.co/cointegrated/rubert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/d9b64e1a24a590ee04161821000abd8b25f6d111e31c013f8016ae9403385b31.fb42ff637f72882f9aa6f1883cdb8ee9e6ba319eadc2daed4f887496cce58ec6
Model config BertConfig {
  "_name_or_path": "cointegrated/rubert-tiny",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.5",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29564
}

loading file https://huggingfa

In [145]:
# encoded_datasets = []
# for tokenizer in tokenizers:
#     encoded_datasets.append(dataset.map(tokenize, batched=True))

encoded_dataset = dataset.map(tokenize, batched=True)

  0%|          | 0/188 [00:00<?, ?ba/s]

  0%|          | 0/81 [00:00<?, ?ba/s]

In [146]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'category', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 187378
    })
    test: Dataset({
        features: ['attention_mask', 'category', 'input_ids', 'label', 'text', 'token_type_ids'],
        num_rows: 80306
    })
})

In [147]:
small_train_dataset = encoded_dataset["train"].shuffle(seed=228).select(range(18737)) 
small_eval_dataset = encoded_dataset["test"].shuffle(seed=228).select(range(8030)) 
# full_train_dataset = encoded_datasets["train"]
# full_eval_dataset = encoded_datasets["test"]

In [148]:
from transformers import TrainingArguments, Trainer
from datasets import load_metric

In [149]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

metric = d.load_metric('accuracy') #, 'precision', 'recall') #'glue','mnli', 'accuracy')

In [158]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,label2id=label2id,id2label=id2label,num_labels =NUM_LABELS)

args = TrainingArguments(
      # "test-glue"
      #"test_trainer",
      output_dir = '/content/drive/MyDrive/fsa_hack',
      evaluation_strategy = "steps",
      learning_rate=2e-5, #2e-5
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=3,
      weight_decay=0.05,
      # load_best_model_at_end=False,
      report_to=None
  )

trainer = Trainer(
      model,
      args,
      train_dataset=small_train_dataset,
      eval_dataset=small_eval_dataset,
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
    )

trainer.train()

loading configuration file https://huggingface.co/cointegrated/rubert-tiny/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/d9b64e1a24a590ee04161821000abd8b25f6d111e31c013f8016ae9403385b31.fb42ff637f72882f9aa6f1883cdb8ee9e6ba319eadc2daed4f887496cce58ec6
Model config BertConfig {
  "_name_or_path": "cointegrated/rubert-tiny",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "3540",
    "1": "8464",
    "2": "9436",
    "3": "9437",
    "4": "9391",
    "5": "3414",
    "6": "2526",
    "7": "5361",
    "8": "9750",
    "9": "1996",
    "10": "8460",
    "11": "9295",
    "12": "3185",
    "13": "3416",
    "14": "2321",
    "15": "5500",
    "16": "2381",
    "17": "3451",
    "18": "2293",
    "19": "9461",
    "20": "5772",
    "21": "

Step,Training Loss,Validation Loss,Accuracy
500,2.665000,1.728558,0.634994
1000,1.537700,1.370373,0.723910
1500,1.343200,1.177301,0.751183
2000,1.138100,1.067318,0.766376
2500,1.040400,1.002544,0.790785
3000,1.013200,0.965751,0.803611
3500,0.993100,0.954505,0.806227


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, text.
***** Running Evaluation *****
  Num examples = 8030
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/fsa_hack/checkpoint-500
Configuration saved in /content/drive/MyDrive/fsa_hack/checkpoint-500/config.json
Model weights saved in /content/drive/MyDrive/fsa_hack/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/fsa_hack/checkpoint-500/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/fsa_hack/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, text.
***** Running Evaluation *****
  Num examples = 8030
  Batch size = 16
Saving model checkpoint to /content/drive/MyDrive/fsa_hack/checkpoint-1000
Configuration

TrainOutput(global_step=3516, training_loss=1.388546163582829, metrics={'train_runtime': 1336.258, 'train_samples_per_second': 42.066, 'train_steps_per_second': 2.631, 'total_flos': 421375758640128.0, 'train_loss': 1.388546163582829, 'epoch': 3.0})

In [159]:
outputs = trainer.predict(small_eval_dataset)
y_pred = outputs.predictions.argmax(1)
set(y_pred)

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: category, text.
***** Running Prediction *****
  Num examples = 8030
  Batch size = 16


{5,
 16,
 18,
 20,
 33,
 44,
 52,
 59,
 64,
 68,
 69,
 70,
 72,
 85,
 86,
 88,
 95,
 99,
 112,
 113}

In [160]:
trainer.save_model('/content/drive/MyDrive/fsa_hack/model3')

Saving model checkpoint to /content/drive/MyDrive/fsa_hack/model3
Configuration saved in /content/drive/MyDrive/fsa_hack/model3/config.json
Model weights saved in /content/drive/MyDrive/fsa_hack/model3/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/fsa_hack/model3/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/fsa_hack/model3/special_tokens_map.json


In [161]:
outputs

PredictionOutput(predictions=array([[-2.9972844 , -2.5739799 , -1.1357272 , ..., -2.5247898 ,
        -2.7487938 , -3.3960698 ],
       [-2.9797466 , -2.5625896 , -1.1343943 , ..., -2.514269  ,
        -2.7349036 , -3.374398  ],
       [-2.9666543 , -2.5636785 , -1.1326365 , ..., -2.5064397 ,
        -2.732606  , -3.359623  ],
       ...,
       [-2.9756734 , -2.55443   , -1.132986  , ..., -2.510294  ,
        -2.7343645 , -3.3644693 ],
       [-2.9867866 , -2.5641654 , -1.1297988 , ..., -2.5234094 ,
        -2.7358358 , -3.397105  ],
       [-1.5687824 , -0.70553887,  0.38767734, ..., -1.3934264 ,
        -0.9943308 , -2.1924672 ]], dtype=float32), label_ids=array([85, 85, 85, ..., 85, 85, 70]), metrics={'test_loss': 0.9544879198074341, 'test_accuracy': 0.8062266500622665, 'test_runtime': 52.8522, 'test_samples_per_second': 151.933, 'test_steps_per_second': 9.498})

In [ ]:
small_eval_dataset

Dataset({
    features: ['attention_mask', 'category', 'input_ids', 'label', 'text', 'token_type_ids'],
    num_rows: 8030
})

# TODO:  
- достать человекопонятные предсказания, причем не одно
- сохранить модель
- суметь скорить любой текст через сохраненную модель

# Полезные ссылки

- https://huggingface.co/transformers/training.html
- https://github.com/huggingface/notebooks/blob/master/transformers_doc/custom_datasets.ipynb
- https://www.kaggle.com/constantinwerner/three-berts-to-solve-intent-classification
- https://discuss.huggingface.co/t/i-have-trained-my-classifier-now-how-do-i-do-predictions/3625 (возможно можно всю логику обернуть через pipe, тогда вроде как предсказания легко разобыть)

In [35]:
from transformers import AutoModel
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
import pymorphy2
import re
tokenizer = AutoTokenizer.from_pretrained('.\model3')
model = AutoModelForSequenceClassification.from_pretrained('.\model3')
inp = pd.DataFrame(columns=['text', 'category'])
inp = inp.append({'text':"Парацетамол таблетки 500 мг 10 шт",'category':'9300'}, ignore_index=True)
inp['text'] = inp['text'].apply(lambda x: re.findall(r'[а-я]+', x.lower()))


def pos(text, morph=pymorphy2.MorphAnalyzer()):
    functors_pos = {'INTJ', 'PRCL', 'CONJ', 'PREP'}
    r = []
    for w in text:
        if w not in functors_pos:
            r.append(w)
    return ' '.join(r)
inp['text'] =inp['text'].apply(pos)
inp

,text,category
0,парацетамол таблетки мг шт,9300


In [39]:

from datasets import load_dataset, Dataset

pred_dataset = Dataset.from_pandas(inp[['text', 'category']])

AttributeError: module 'huggingface_hub.hf_api' has no attribute 'DatasetInfo'

In [31]:
inputs = tokenizer(pred_dataset, return_tensors="pt", padding=True, truncation=True)
generation_output = model.generate(**inputs, return_dict_in_generate=True, output_scores=True)

NameError: name 'pred_dataset' is not defined

In [19]:
inputs

{'input_ids': [2, 14892, 1232, 2550, 15585, 4375, 845, 18814, 1588, 954, 1029, 26068, 575, 336, 751, 3, 16110, 1140, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}